# Join datasets to GrMix dataset

catinfo created in train procedure and is the same for all other procedures.


In [70]:
from pycocotools.coco import COCO
from grcore.coco.dataset import CocoDataset
from grcore.coco import ann, init_dataset_folder
from grcore.coco.file_io import save_cocodict
from pathlib import Path
import shutil
import copy

In [71]:
porw = "wholes"

In [72]:
base_outpath = Path("/data/dataset/grmix/orig")
init_dataset_folder(base_outpath)

In [73]:
inforecord = ann.build_info_info("GrMix dataset in COCO format", contributor="Gregor Koporec", version="v1.0")
licenserecords = [ann.build_lic_info(1, "Gorenje Copyright", "./LICENSE")]

## Train

In [74]:
base_personpath = Path(f"/data/dataset/pascal/pascalparts_person")
personpath = base_personpath / f"annotations/train_{porw}.json"
base_carpath = Path(f"/data/dataset/car-parts-segmentation/ocln")
carpath = base_carpath / f"annotations/train_{porw}.json"

In [75]:
persondset = COCO(str(personpath))
cardset = COCO(str(carpath))

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


### Cat

In [76]:
catrecords = []
personcats = persondset.cats.values()
carcats = cardset.cats.values()

catmap = {"person": {}, "car":{}}
id_ = 1
for cat in personcats:
    catmap["person"][cat["id"]] = id_
    cat["id"] = id_
    catrecords.append(cat)
    id_ += 1
    

for cat in carcats:
    catmap["car"][cat["id"]] = id_
    cat["id"] = id_
    catrecords.append(cat)
    id_ += 1

### Imgs

In [77]:
imgrecords = []
personimgs = persondset.imgs.values()
carimgs = cardset.imgs.values()

imgmap = {"person":{}, "car":{}}
id_ = 1
for img in personimgs:
    imgmap["person"][img["id"]] = id_
    img["id"] = id_
    imgrecords.append(img)
    id_ += 1

for img in carimgs:
    imgmap["car"][img["id"]] = id_
    img["id"] = id_
    imgrecords.append(img)
    id_ += 1

In [78]:
# Copy images into new folders
for imgrecord in imgrecords:
    if imgrecord["dataset"] == "Car Parts":
        basepath = base_carpath
    else:
        basepath = base_personpath

    src = basepath / imgrecord["coco_url"]
    dst = base_outpath / imgrecord["coco_url"]
    if not dst.exists():
        shutil.copy(src, dst)

### Anns

In [79]:
annrecords = []
personanns = persondset.anns.values()
caranns = cardset.anns.values()

id_ = 1
for anno in personanns:
    anno["id"] = id_
    anno["image_id"] = imgmap["person"][anno["image_id"]]
    anno["category_id"] = catmap["person"][anno["category_id"]]
    annrecords.append(anno)
    id_ += 1

for anno in caranns:
    anno["id"] = id_
    anno["image_id"] = imgmap["car"][anno["image_id"]]
    anno["category_id"] = catmap["car"][anno["category_id"]]
    annrecords.append(anno)
    id_ += 1

### Save train

In [80]:
cocodict = ann.build_dset_info(inforecord, licenserecords, catrecords, imgrecords, annrecords)
outpath = base_outpath / "annotations" /  f"train_{porw}.json"
save_cocodict(cocodict, outpath)

Saving json: /data/dataset/grmix/orig/annotations/train_wholes.json


## Val

In [81]:
base_personpath = Path(f"/data/dataset/pascal/pascalparts_person")
personpath = base_personpath / f"annotations/val_{porw}_subset.json"
base_carpath = Path(f"/data/dataset/car-parts-segmentation/ocln")
carpath = base_carpath / f"annotations/val_{porw}.json"

In [82]:
persondset = COCO(str(personpath))
cardset = COCO(str(carpath))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


### Imgs


In [83]:
imgrecords = []
personimgs = persondset.imgs.values()
carimgs = cardset.imgs.values()

imgmap = {"person":{}, "car":{}}
id_ = 1
for img in personimgs:
    imgmap["person"][img["id"]] = id_
    img["id"] = id_
    imgrecords.append(img)
    id_ += 1

for img in carimgs:
    imgmap["car"][img["id"]] = id_
    img["id"] = id_
    img["coco_url"] = f"val/{img['file_name']}"
    imgrecords.append(img)
    id_ += 1

In [84]:
# Copy images into new folders
for imgrecord in imgrecords:
    if imgrecord["dataset"] == "Car Parts":
        basepath = base_carpath
    else:
        basepath = base_personpath

    src = basepath / imgrecord["coco_url"]
    dst = base_outpath / imgrecord["coco_url"]
    if not dst.exists():
        shutil.copy(src, dst)

### Annos

In [85]:
annrecords = []
personanns = persondset.anns.values()
caranns = cardset.anns.values()

id_ = 1
for anno in personanns:
    anno["id"] = id_
    anno["image_id"] = imgmap["person"][anno["image_id"]]
    anno["category_id"] = catmap["person"][anno["category_id"]]
    annrecords.append(anno)
    id_ += 1

for anno in caranns:
    anno["id"] = id_
    anno["image_id"] = imgmap["car"][anno["image_id"]]
    anno["category_id"] = catmap["car"][anno["category_id"]]
    annrecords.append(anno)
    id_ += 1

### Save

In [86]:
cocodict = ann.build_dset_info(inforecord, licenserecords, catrecords, imgrecords, annrecords)
outpath = base_outpath / "annotations" / f"val_{porw}.json"
save_cocodict(cocodict, outpath)

Saving json: /data/dataset/grmix/orig/annotations/val_wholes.json


## Test

In [87]:
base_personpath = Path(f"/data/dataset/coco/coco_ocln/")
personpath = base_personpath / "annotations/val_wholes.json"
base_carpath = Path(f"/data/dataset/car-parts-segmentation/ocln")
carpath = base_carpath / "annotations/test_wholes.json"

In [88]:
persondset = COCO(str(personpath))
cardset = COCO(str(carpath))

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [89]:
imgrecords = []
personimgs = persondset.imgs.values()
carimgs = cardset.imgs.values()

imgmap = {"person":{}, "car":{}}
id_ = 1
for img in personimgs:
    imgmap["person"][img["id"]] = id_
    img["id"] = id_
    img["coco_url"] = f"test/{img['file_name']}"
    imgrecords.append(img)
    id_ += 1

for img in carimgs:
    imgmap["car"][img["id"]] = id_
    img["id"] = id_
    imgrecords.append(img)
    id_ += 1

In [90]:
# Copy images into new folders
for imgrecord in imgrecords:
    if imgrecord["dataset"] == "Car Parts":
        basepath = base_carpath
        src = basepath / imgrecord["coco_url"]
    else:
        basepath = base_personpath
        src = basepath / "val" / imgrecord["file_name"]

    dst = base_outpath / imgrecord["coco_url"]
    if not dst.exists():
        shutil.copy(src, dst)

In [91]:
annrecords = []
personanns = persondset.anns.values()
caranns = cardset.anns.values()

id_ = 1
for anno in personanns:
    anno["id"] = id_
    anno["image_id"] = imgmap["person"][anno["image_id"]]
    anno["category_id"] = catmap["person"][anno["category_id"]]
    annrecords.append(anno)
    id_ += 1

for anno in caranns:
    anno["id"] = id_
    anno["image_id"] = imgmap["car"][anno["image_id"]]
    anno["category_id"] = catmap["car"][anno["category_id"]]
    annrecords.append(anno)
    id_ += 1

In [92]:
cocodict = ann.build_dset_info(inforecord, licenserecords, catrecords, imgrecords, annrecords)
outpath = base_outpath / "annotations" / f"test_wholes.json"
save_cocodict(cocodict, outpath)

Saving json: /data/dataset/grmix/orig/annotations/test_wholes.json
